In [ ]:
#Configuração global
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from PIL import Image
from tqdm import tqdm

from torchvision import datasets, transforms
from torchvision.models import densenet121, DenseNet121_Weights
from torch.utils.data import DataLoader, Subset

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

#Reprodutibilidade (Definição da seed)
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

#Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
#SPLIT TREINO / TESTE INTERNO

data_dir = "data/train"

full_dataset = datasets.ImageFolder(root=data_dir)

targets = full_dataset.targets

train_idx, test_idx = train_test_split(
    range(len(targets)),
    test_size=0.2,
    stratify=targets,
    random_state=42
)

train_dataset_full = Subset(full_dataset, train_idx)
test_dataset_internal = Subset(full_dataset, test_idx)

print("Treino:", len(train_dataset_full))
print("Teste interno:", len(test_dataset_internal))

In [ ]:
#Train transform (treinamento)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

#Train transofrm (validação)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
#DIVISÃO TREINO/VALIDAÇÃO

#Dataset base (sem transform)
dataset = train_dataset_full

targets = np.array(dataset.dataset.targets)[dataset.indices]

#Split estratificado
train_idx, val_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.2,
    stratify=targets,
    random_state=42
)

#Criar datasets separados com transform definido
train_dataset = Subset(
    datasets.ImageFolder(root=data_dir, transform=train_transform),
    train_idx
)

val_dataset = Subset(
    datasets.ImageFolder(root=data_dir, transform=val_transform),
    val_idx
)

In [ ]:
#DATALOADERS

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
#DEFINIÇÃO DO MODELO

#Inicialização do modelo (transfer learning)
weights = DenseNet121_Weights.DEFAULT
model = densenet121(weights=weights)

#Ajuste da camada final para classificação binária
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 1)

#Envio do modelo para o device
model = model.to(device)

In [ ]:
#FUNÇÃO DE PERDA E OTIMIZADOR

train_targets = np.array(targets)[train_idx]

num_pneumonia = np.sum(train_targets == 1)
num_normal = np.sum(train_targets == 0)

pos_weight = torch.tensor(
    [num_normal / num_pneumonia],
    dtype=torch.float32
).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
#FUNÇÕES DE TREINAMENTO E VALIDAÇÃO

#Função de treino
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()

    running_loss = 0.0

    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(loader.dataset)

    return epoch_loss

#Função de validação
def validate(model, loader, criterion):
    model.eval()

    running_loss = 0.0
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            probs = torch.sigmoid(outputs)

            running_loss += loss.item() * images.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    epoch_loss = running_loss / len(loader.dataset)

    val_auc = roc_auc_score(all_labels, all_probs)

    return epoch_loss, val_auc

In [ ]:
#TREINAMENTO DO MODELO

epochs = 5
best_auc = 0.0

for epoch in range(epochs):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_auc = validate(model, val_loader, criterion)

    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val ROC-AUC: {val_auc:.4f}")

    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), "best_model.pth")
        print("Modelo salvo!")

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
#AVALIAÇÃO FINAL (Teste Interno)

#Data set teste
test_dataset = test_dataset_internal
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Total imagens teste:", len(test_dataset))
print("Classes:", test_dataset.classes)

In [ ]:
#Carregando melhor modelo
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()

In [ ]:
#Avaliação no teste
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        outputs = model(images)
        probs = torch.sigmoid(outputs)

        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

test_auc = roc_auc_score(all_labels, all_probs)

print("Test ROC-AUC:", round(test_auc, 4))

In [ ]:
# Converter para classe binária (threshold 0.5)
preds = (np.array(all_probs) > 0.5).astype(int)

print("\nMatriz de Confusão:")
print(confusion_matrix(all_labels, preds))

print("\nRelatório de Classificação:")
print(classification_report(all_labels, preds))

In [ ]:
#INFERÊNCIA

#Configurações
test_csv_path = "data/test.csv"
test_images_path = "data/test_images"
model_path = "best_model.pth"  # caminho do seu modelo salvo

#Carregar modelo
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

#Tranform (igual validação)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

#Ler CSV
test_df = pd.read_csv(test_csv_path)

predictions = []

#Inferência
with torch.no_grad():
    for img_name in tqdm(test_df["id"]):
        img_path = os.path.join(test_images_path, img_name)

        image = Image.open(img_path).convert("RGB")
        image = test_transform(image).unsqueeze(0).to(device)

        output = model(image)
        prob = torch.sigmoid(output).item()

        predictions.append(prob)

#Gerar submissão
submission = pd.DataFrame({
    "id": test_df["id"],
    "target": predictions
})

submission.to_csv("submission.csv", index=False)

print("submission.csv criado com sucesso!")